In [1]:
"""
Pranav Lodha
Sources/ Learning Tools
https://keras.io
https://www.analyticsvidhya.com/blog/2017/06/transfer-learning-the-art-of-fine-tuning-a-pre-trained-model/
https://github.com/rajatvikramsingh/stl10-vgg16/blob/master/vgg_transfer.py
https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html
https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6
"""

# Imports 
from keras.models import Sequential
from scipy.misc import imread
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.engine import Model
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
import stl10_input

c:\users\prana\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [2]:
# Download the data required
stl10_input.download_and_extract()

In [3]:
# Store Data
x_train = stl10_input.read_all_images('data/stl10_binary/train_X.bin')
y_train = stl10_input.read_labels('data/stl10_binary/train_y.bin')
x_test = stl10_input.read_all_images('data/stl10_binary/test_X.bin')
y_test = stl10_input.read_labels('data/stl10_binary/test_y.bin')
# unlabeled = stl10_input.read_all_images('data/stl10_binary/unlabeled_X.bin')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
# print(unlabeled.shape)

(5000, 96, 96, 3)
(5000,)
(8000, 96, 96, 3)
(8000,)


In [4]:
# Model
model = VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3), classes=10)

# Model Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0         
__________

In [5]:
# Compile the model
# model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Add to the Model for increased accuracy
last = model.output
x = Flatten()(last)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
pred = (Dense(10, activation='sigmoid'))(x)
model = Model(model.input, pred)
# model.add(Dense(10, activation='softmax'))

sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

In [6]:
# Freeze Layers of VGG16
for layer in model.layers[:20]:
    layer.trainable = False

In [7]:
# Was having array issue's so this should fix
y_train = keras.utils.to_categorical(y_train - 1, num_classes =  10)
y_test = keras.utils.to_categorical(y_test - 1, num_classes = 10)

model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0         
__________

In [8]:
# Fit my training data to the model
model.fit(x_train, y_train, epochs=5, batch_size=32)
# model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data = (x_train, y_train))

Epoch 1/5
5000/5000 [==============================] - 910s 182ms/step - loss: 0.6531 - acc: 0.5952
Epoch 2/5
5000/5000 [==============================] - 952s 190ms/step - loss: 0.5960 - acc: 0.6968
Epoch 3/5
5000/5000 [==============================] - 911s 182ms/step - loss: 0.5629 - acc: 0.7938
Epoch 4/5
5000/5000 [==============================] - 909s 182ms/step - loss: 0.5346 - acc: 0.8994
Epoch 5/5
5000/5000 [==============================] - 911s 182ms/step - loss: 0.5103 - acc: 0.9000


In [9]:
# Percent lost and percent correct
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)
print("X-Test and Y-Test: " , loss_and_metrics)

8000/8000 [==============================] - 1462s 183ms/step
X-Test and Y-Test:  [0.4992568515539169, 0.8998500363826751]


In [10]:
# Predict the y values of the unlabeled data set
# classes = model.predict(unlabeled, batch_size=32)
# print(classes)

# Write the data to a file, to loop at later.
# f = open( 'classes.txt', 'w' )
# f.write( 'dict = ' + repr(classes) + '\n' )
# f.close()

# loss_and_metrics = model.evaluate(unlabeled, classes, batch_size = 32)
# print("Unlabeled: " , loss_and_metrics)